In [1]:
import torch
import random
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForSequenceClassification
path = "../model/Reranker"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSequenceClassification.from_pretrained(path)

In [2]:
from utils import Data
data_processor = Data(root_path="../")
data_processor.read_in_memory()
data_processor.dataset.keys()

正在处理文件collection.sampled.tsv 读取文件的格式为('pid', 'passage')
正在处理文件train_sample_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件train_sample_passv2_qrels.tsv 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件val_2021_53_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件val_2021_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件val_2021.qrels.pass.final.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')
正在处理文件test_2022_76_queries.tsv 读取文件的格式为('qid', 'query')
正在处理文件test_2022_passage_top100.txt 读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
正在处理文件test_2022.qrels.pass.withDupes.txt 读取文件的格式为('qid', 'mark', 'pid', 'rating')


dict_keys(['collection.sampled', 'train_sample_queries', 'train_sample_passv2_qrels', 'val_2021_53_queries', 'val_2021_passage_top100', 'val_2021.qrels.pass.final', 'test_2022_76_queries', 'test_2022_passage_top100', 'test_2022.qrels.pass.withDupes'])

In [3]:
random_query_id = random.choice(tuple(data_processor.dataset["train_sample_queries"].keys()))
random_query = data_processor.dataset["train_sample_queries"][random_query_id]["query"]
random_query

'what is personal essence'

In [4]:
answers = {
    v['rating']: data_processor.dataset["collection.sampled"][pid]['passage']
    for pid, v in data_processor.dataset["train_sample_passv2_qrels"].items()
    if v["qid"] == random_query_id
}
len(answers)

1

In [5]:
print("Query:", random_query)
print("Answers:", answers['1'])

Query: what is personal essence
Answers: 'Concepts. When the personal essence is strong and in harmony with the individual there is health. Imbalance, lack of harmony or coherence in the personal essence is a precursor (even an actual cause) of subsequent disease.'


In [6]:
inputs = tokenizer(
    random_query,
    answers['1'],
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)
inputs

{'input_ids': tensor([[  101,  2054,  2003,  3167, 11305,   102,  1005,  8474,  1012,  2043,
          1996,  3167, 11305,  2003,  2844,  1998,  1999,  9396,  2007,  1996,
          3265,  2045,  2003,  2740,  1012, 10047, 26657,  1010,  3768,  1997,
          9396,  2030,  2522,  5886, 10127,  1999,  1996,  3167, 11305,  2003,
          1037, 14988,  1006,  2130,  2019,  5025,  3426,  1007,  1997,  4745,
          4295,  1012,  1005,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [7]:
with torch.no_grad():
    outputs = model(**inputs)
outputs.logits

tensor([[3.6677]])